# IPM DSS

## import modules

In [2]:
import pandas

from openalea.dss import Hub, Model



## Access to DSS catalog

In [ ]:
hub = Hub()
#hub.list_dss(ViewDataFrame=True)
print(hub) 

hub.display(ViewDataFrame=False) #to improve

In [ ]:
hub.list_dss(View)


## Access to one DSS

In [ ]:
#PSI=hub.get_dss(ModelId='no.nibio.vips', DSSId='PSILARTEMP') 
# hub[dss][model]
PSI=hub.get_model(dss="no.nibio.vips", model='PSILARTEMP')
print(PSI)# PSILARTEMP

In [ ]:
# 
weather_data = ...
PSI(weather_data)
PSI.input_DSS_weather_model_json(TimeStart="2021-01-01", 
                                 TimeEnd="2021-12-31",
                                 weatherDataService='Finnish Meteorological Institute measured data',
                                 parameters=[1002],
                                 stationId=[101104])

In [ ]:
ds=PSI.get_data_model(modelInput='model_input_weatherdata.json')
ds

In [ ]:
ds.to_dataframe().plot.line()

In [ ]:
PSILAROBS= hub.get_dss(DSSId='PSILAROBSE')


In [ ]:
PSILAROBS.input_DSS_fieldobservation_model_json(fieldobservation=r"C:\Users\mlabadie\Documents\GitHub\dss\src\openalea\dss\fieldobservation.json")

In [ ]:
PSILAROBS.get_data_model(modelInput="model_input_fieldobservation.json")